In [5]:
%load_ext autoreload
%autoreload 2

from statsmodels.base.model import GenericLikelihoodModel

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats
import sys

sys.path.append("../")
import vuong_tests5

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
class OLS_loglike(GenericLikelihoodModel):
    
    def __init__(self, *args,ols=False, **kwargs):
        super(OLS_loglike,self).__init__(*args,**kwargs)
        self.ols = ols

    def loglikeobs(self, params):
        y = self.endog
        x = self.exog
        mu_y = np.matmul(x,params)  
        resid = y - mu_y
        sigma = np.sqrt(np.sum(resid**2)/resid.shape[0])
        pr_y = stats.norm.logpdf( resid, loc=0,scale=sigma )
        return pr_y


def setup_shi(yn,xn,return_model=False,num_params=4):
    x1n,x2n = xn[:,0],xn[:,1:num_params]
    
    # model 1 grad, etc.
    model1 = sm.OLS(yn,sm.add_constant(x1n))
    model1_fit = model1.fit(disp=False)
    params1 = (model1_fit.params)
    
    model1_deriv = OLS_loglike(yn,sm.add_constant(x1n))
    ll1 = model1_deriv.loglikeobs(model1_fit.params)
    grad1 =  model1_deriv.score_obs(model1_fit.params)    
    hess1 = model1_deriv.hessian(model1_fit.params)
    
    #model 2 grad, etc.
    model2 = sm.OLS(yn,sm.add_constant(x2n))
    model2_fit = model2.fit(disp=False)
    params2 = (model2_fit.params)
    
    model2_deriv = OLS_loglike(yn,sm.add_constant(x2n))
    ll2 = model2_deriv.loglikeobs(model2_fit.params)
    grad2 =  model2_deriv.score_obs(model2_fit.params)    
    hess2 = model2_deriv.hessian(model2_fit.params)
    
    if return_model:
        return ll1,grad1,hess1,params1,model1,ll2,grad2,hess2,params2,model2
    return ll1,grad1,hess1,params1,ll2,grad2,hess2,params2

def gen_data(nobs=1000, a=0.25, num_params=4):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a*x[:,0] + a/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

yn,xn,nobs = gen_data()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn,return_model=False,num_params=15)

# a  = .125, something weird here too...

In [7]:
a = .125
num_params = 19

In [8]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.99 & 1.00 & 0.93 & 0.97 & 0.94 & 0.99   \\
Model 1 & 0.01 & 0.00 & 0.07 & 0.03 & 0.06 & 0.01   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.99, 0.01, 0.  ]), array([1., 0., 0.]), array([0.93, 0.07, 0.  ]), array([0.97, 0.03, 0.  ]), array([0.94, 0.06, 0.  ]), array([0.99, 0.01, 0.  ]), 0.6502316847425798, 2.7383816840804545, 2.8140570450423663)


In [9]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.98 & 1.00 & 0.92 & 0.92 & 0.91 & 0.98   \\
Model 1 & 0.02 & 0.00 & 0.08 & 0.08 & 0.09 & 0.02   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.98, 0.02, 0.  ]), array([1., 0., 0.]), array([0.92, 0.08, 0.  ]), array([0.92, 0.08, 0.  ]), array([0.91, 0.09, 0.  ]), array([0.98, 0.02, 0.  ]), 2.346990255578465, 3.259494023542413, 3.4981419106006633)


In [10]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=1000, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(100,gen_data_ex,setup_shi)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.88 & 1.00 & 0.79 & 0.82 & 0.79 & 0.88   \\
Model 1 & 0.12 & 0.00 & 0.21 & 0.18 & 0.21 & 0.12   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.88, 0.12, 0.  ]), array([1., 0., 0.]), array([0.79, 0.21, 0.  ]), array([0.82, 0.18, 0.  ]), array([0.79, 0.21, 0.  ]), array([0.88, 0.12, 0.  ]), 5.027217188614881, 4.46991139421584, 4.5078855782550935)


# a = 0

In [11]:
a = 0
num_params = 19

In [12]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(100,gen_data_ex,setup_shi)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 1.00 & 1.00 & 0.99 & 0.99 & 0.99 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.01 & 0.01 & 0.01 & 0.00   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([1., 0., 0.]), array([1., 0., 0.]), array([0.99, 0.01, 0.  ]), array([0.99, 0.01, 0.  ]), array([0.99, 0.01, 0.  ]), array([1., 0., 0.]), -1.0343453085790264, 1.3465111237578915, 1.8455663448782533)


In [13]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(100,gen_data_ex,setup_shi)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 1.00 & 1.00 & 1.00 & 1.00 & 1.00 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), -0.9569365549146234, 1.5158458596826496, 1.8572443821196896)


In [14]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=1000, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(100,gen_data_ex,setup_shi)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 1.00 & 1.00 & 1.00 & 1.00 & 1.00 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), -0.931831221776922, 1.5301426579279638, 1.948134753256252)
